In [34]:
import random
from time import sleep
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import sys
import numpy

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\ramir\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


Hites

In [35]:
url = "https://www.hites.com/tecnologia/computacion/notebook/?start=0&sz=70"
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data,"html.parser")
bloque = soup.find_all("div",{"class":"col-6 col-lg-4 px-0"})

In [36]:
nombres = list()
precioaux = list()
precio = list()
codigos = list()
modelo = list()
encontrarIndiceModelo = list()
encontrarindice = list()
caracteristica = list()
almacenamientoaux = list()
almacenamiento = list()
procesadoraux = list()
procesador = list()
ramaux = list()
ram = list()
precios = list()

for i in bloque:
    nombre = i.find("a",{"class":"link product-name"})
    caract = i.find_all("li",{"class":"attribute-values"})
    precios = i.find("span",{"class":"value"})
    codigo = i.find("span",{"class":"d-block product-sku"})
    nombres.append(nombre.text)
    caracteristica.append(caract)  
    precioaux.append(precios.text)
    codigos.append(codigo.text)

""""Obtener cada elemento de almacenamiento, memoria ram y procesador definiendo un valor auxiliar 
    para almacenamiento en caso de no encontrarse en la pagina"""
for i in range(len(caracteristica)):
    if (len(caracteristica[i])==2):
        almacenamientoaux.append(caracteristica[i][0])
        ramaux.append(caracteristica[i][0])
        procesadoraux.append(caracteristica[i][1])
        
    else:
        almacenamientoaux.append(caracteristica[i][0])
        ramaux.append(caracteristica[i][1])
        procesadoraux.append(caracteristica[i][2])

#Insertar elementos de procesador y memoria ram en las listas como texto, en el caso de almacenamiento como str
for i in range(len(almacenamientoaux)):
    if(type(almacenamientoaux[i]==str)):
        almacenamiento.append(almacenamientoaux[i])
        procesador.append(procesadoraux[i].text)
        ram.append(ramaux[i].text)
        
    else:
        almacenamiento.append(almacenamientoaux[i].text)
        procesador.append(procesadoraux[i].text)
        ram.append(ramaux[i].text)  

#Modificar los elementos de almacenamiento a formato text
for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].text
    

#Modicar valor auxiliar de almacenamiento en caso de que no se encontrará en la pagina          
for i in range(len(almacenamiento)):
    if ('RAM' in almacenamiento[i])==True:
        almacenamiento[i]="Almacenamiento no especificado"
        
#Reemplazar los saltos de linea del formato text a expacios en blanco para obtener solo los datos deseados
for i in range(len(caracteristica)):
    almacenamiento[i] = almacenamiento[i].replace('\n','')
    ram[i] = ram[i].replace('\n','')
    procesador[i] = procesador[i].replace('\n','')

#Encontrar el indice del primer slash para poder separar el nombre original sin los componentes
for i in range(len(nombres)):
    encontrarindice.append(nombres[i].find("/"))

#Dejar solo el nombre de los notebooks dentro de la lista
for i in range(len(nombres)):
    nombres[i] = nombres[i][0:encontrarindice[i]-1]
    
#Eliminación de caracteres especiales
for i in range(len(precioaux)):
    precioaux[i] = precioaux[i].replace("Price reduced from", "")
    precioaux[i] = precioaux[i].replace("\n","")
    precioaux[i] = precioaux[i].replace("to","")
    precioaux[i] = precioaux[i].replace("$","")
    precioaux[i] = precioaux[i].replace(".","")
    precioaux[i] = precioaux[i].replace("Normal","")
    precioaux[i] = precioaux[i].replace("Oferta","")
    precio.append(precioaux[i])
    
for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].replace("Almacenamiento:", "")
    
for i in range(len(procesador)):
    procesador[i] = procesador[i].replace("Procesador:", "")
    
for i in range(len(ram)):
    ram[i] = ram[i].replace("Memoria RAM:", "")

for i in range(len(codigos)):
    codigos[i] = codigos[i].replace("Código: ", "")
    codigos[i] = codigos[i].replace("\n","")  
    
#Encontrar el indice para el modelo

for i in range(len(nombres)):
    encontrarIndiceModelo.append(nombres[i].find(" "))
    
for i in range(len(nombres)):
    modelo.append(nombres[i][encontrarIndiceModelo[i]:len(nombres[i])-1])
    
for i in range(len(modelo)):
    modelo[i] = modelo[i][1:len(nombres)-1]

In [44]:

df = pd.DataFrame(columns={'Nombre', 'Codigo', 'Modelo', 'Almacenamiento', 'Procesador', 'Memoria Ram', 'Precio', 'Fecha'})
x = datetime.today().strftime('%Y-%m-%d')


TypeError: __init__() got an unexpected keyword argument 'axis'

In [41]:
for i in range(len(caracteristica)):
    df = df.append({'Nombre': nombres[i], 'Codigo':codigos[i], 'Modelo':modelo[i] ,'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], 'Precio':precio[i], 'Fecha':x},ignore_index=True)

In [5]:
modelo

['Reacondicionado Hp 14-fq0013d',
 'Lenovo Ideapad 3 14alc',
 'Gamer Asus Tuf Gaming Fx506lh-hn082',
 'Lenovo Ideapad ',
 'Asus X515ea-bq968',
 'Asus X515ma-br576',
 'Reacondicionado Hp 14-dq1043c I',
 'Asus X415ea-eb742',
 'Hp 14-ck2091l',
 'Hp Pavilion X360 Convertible 14-dy0002l',
 'Asus M415da-ek844',
 'Gamer Lenovo Ideapad Gaming 3 15ARH0',
 'Hp 240 G',
 'Hp 240 G',
 'Hp 15-ef1018l',
 'Hp Pavilion 17-by3063st Reacondicionad',
 'Hp 15-cs3153cl Reacondicionad',
 'Lenovo Yoga Slim ',
 'Lenovo Yoga Slim 9 14itl',
 'p 14a-na0010ca Reacondicionad',
 'Lenovo Ideapad S14',
 'Hp Pavilion Gaming Laptop 15 Dk1028l',
 'Gamer Acer Nitro ',
 'Lenovo Ideapad Flex ',
 'Lenovo Ideapad 3 15itl',
 'Lenovo Ideapad 3 14itl',
 'Lenovo Yoga 9 14itl',
 'Lenovo Ideapad ',
 'Msi Prestige 14evo A11m-250c',
 'Hp 348 G',
 'Reacondicionado Hp 14m-dw1013d',
 'Lenovo Ideapad S34',
 'Lenovo Ideapad S145-14ii',
 'Reacondicionado Hp 15-da0022d',
 'Lenovo Ideapad S14',
 'Asus L41',
 'Ai',
 'Gamer Hp Pavilion 15-dk10

PC FACTORY

In [6]:
url_base=[]
url_notebooks=[]
url1 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636"
url2 = "https://www.pcfactory.cl/notebooks?categoria=735&papa=636&pagina=2"

url_base.append(url1)
url_base.append(url2)


for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="product"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)


In [7]:
marca = []
nombres = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigos = []
ram = []
precios = []

for i in url_notebooks:
    maux = 0
    moaux = 0
    almaux = 0
    proaux = 0
    ramaux = 0
    driver.get(i)
    bloque_codigos = driver.find_elements_by_xpath('//p[@class="link color-dark"]')

    codigo = bloque_codigos[1]
    codigos.append(codigo.text)

    precio = driver.find_element_by_xpath('//div[@class="price-xl color-primary-1"]').text
    precios.append(precio)

    
    
    caracteristicas = driver.find_elements_by_xpath("//li")

    for i in range(len(caracteristicas)):
        caracteristicas[i] = caracteristicas[i].text

    for i in caracteristicas:
        if(i.find("Marca")!=-1):
            maux = maux + 1
            nombre_marca = i
        if(i.find("Modelo")!=-1):
            moaux = moaux + 1
            nombre_modelo = i
        if(i.find("Almacenamiento")!=-1):
            almaux = almaux + 1
            nombre_almacenamiento = i
        if(i.find("RAM:")!=-1 or i.find("Memoria:")!=-1 or i.find("Memoria Total:")!=-1):
            ramaux = ramaux + 1
            nombre_ram = i
        if(i.find("Procesador:")!=-1):
            proaux = proaux + 1
            nombre_procesador = i

    if(maux >= 1):
        marca.append(nombre_marca)
    elif(maux == 0):
        marca.append("Marca no encontrada")
    if(moaux >= 1):
        modelo.append(nombre_modelo)
    elif(moaux == 0):
        modelo.append("Modelo no encontrado")
    if(almaux >= 1):
        almacenamiento.append(nombre_almacenamiento)
    elif (almaux == 0 ):
        almacenamiento.append("Almacenamiento no encontrado")
    if(ramaux >= 1):
        ram.append(nombre_ram)
    elif(ramaux == 0):
        ram.append("Memoria ram no encontrada")
    if(proaux >= 1):
        procesador.append(nombre_procesador)
    elif(proaux == 0):
        procesador.append("Procesador no encontrado")

    sleep(random.uniform(2,4))



for i in range(len(almacenamiento)):
    almacenamiento[i] = almacenamiento[i].replace("\xa0", " ")
    almacenamiento[i] = almacenamiento[i].replace("Almacenamiento:", "")
    almacenamiento[i] = almacenamiento[i].lstrip()

for i in range(len(marca)):
    marca[i] = marca[i].replace("\xa0"," ")
    marca[i] = marca[i].replace("Marca:","")
    marca[i] = marca[i].lstrip()

for i in range(len(modelo)):
    modelo[i] = modelo[i].replace("\xa0", " ")
    modelo[i] = modelo[i].replace("Modelo:", "")
    modelo[i] = modelo[i].lstrip()

for i in range(len(ram)):
    ram[i] = ram[i].replace("\xa0", " ")
    ram[i] = ram[i].replace("Memoria RAM:", "")
    ram[i] = ram[i].replace("Memoria:", "")
    ram[i] = ram[i].replace("Memoria Total:", "")
    ram[i] = ram[i].lstrip()

for i in range(len(procesador)):
    procesador[i] = procesador[i].replace("\xa0"," ")
    procesador[i] = procesador[i].replace("Procesador:","")
    procesador[i] = procesador[i].lstrip()

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ","")
    precios[i] = precios[i].replace("$","")
    precios[i] = precios[i].replace(".","")

for i in range(len(marca)):
    nombres.append(marca[i] + " " + modelo[i])

for i in range(len(codigos)):
    codigos[i] = codigos[i].replace("ID ","")

x = datetime.today().strftime('%Y-%m-%d')

for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombres[i],'Codigo':codigos[i],'Modelo':modelo[i] ,'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], 'Fecha':x},ignore_index=True)

In [8]:
modelo

['Chromebook',
 '240 G7',
 'X515MA-BR576T',
 'X515MA-BR288T',
 'X543UA-DM3470T',
 '250 G8',
 'Vostro 3400',
 'HP 348 G7',
 '240 G7',
 'X412FA-EB1257T',
 '14-dq2026la',
 '15-dy2064la',
 'Pavilion x360',
 'X515EA-BQ507T',
 '14-dy0501la',
 'Aspire 3 A314-22-R56E-1',
 'Inspiron 3501',
 'Matebook D 15',
 'Matebook D 14',
 'Pavilion Aero 13-be0501la',
 'ideapad S540-13IML',
 'Matebook D 14',
 '15-eh0011la',
 'ideapad S540-13IML',
 'X571GT-BQ930T',
 'Nitro 5 AN515-55-56P2-2',
 'Nitro 5 AN515-54-59UV-7',
 'Inspiron ',
 'D433IA-EB934R',
 'Inspiron 5310',
 '15-ec1029la',
 'Macbook Air',
 'Macbook Air',
 'Macbook Air',
 'FX506HC-HN042T',
 'Inspiron 15 5502',
 '13-ba1123la',
 'ROG Strix G15 G512LI-HN061T',
 'Ideapad s540',
 'G5 5510',
 'Macbook Air',
 'Macbook Air',
 'Macbook Air',
 'UX435EG-AI094TS',
 'FX516PC-HN021T',
 'G5 5510',
 'XENIA 14',
 'Macbook Pro',
 'Macbook Pro',
 'Matebook X',
 'ThinkPad X13',
 'Mate Xpro',
 'Macbook Pro',
 'Macbook Pro',
 'Yoga 9 14ITL5',
 'Macbook Pro',
 'ThinkPad 

Falabella

In [11]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,11):
    url = "https://www.falabella.com/falabella-cl/category/cat70057/Notebooks?page="+str(i)
    url_base.append(url)


##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="jsx-4001457643 search-results-list"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)


marca = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigo = []
ram = []
precios = []
for i in url_notebooks:
    nombre = []
    valor = []
    driver.get(i)
    sleep(2)
    boton = driver.find_element_by_xpath('//button[@class="jsx-3459521287 mkp-swatchButton mkp-swatchButton-collapseButton"]').click()
    sleep(random.uniform(4,5))
    nombre_aux = driver.find_element_by_xpath('//div[@class="jsx-3686231685 product-name fa--product-name"]').text
    elementos = driver.find_elements_by_xpath('//tr[@class="jsx-428502957"]')
    try:
        precio = driver.find_element_by_xpath('//span[@class="copy12 primary  jsx-340449923 normal    "]').text
        
    except:
        try:
            precio = driver.find_element_by_xpath('//span[@class="copy10 primary  jsx-340449923 normal    "]').text
        except:
            try:
                precio = driver.find_element_by_xpath('//h2[@class="jsx-2034229452 title6 mkp-secondary    "]').text
            except:
                try:
                    precio = driver.find_element_by_xpath('//span[@class="copy12 primary  jsx-2612542277 normal     "]').text
                except:
                    precio = driver.find_element_by_xpath('//span[@class="copy10 primary  jsx-2612542277 normal     "]').text
    codigos = driver.find_element_by_xpath('//span[@class="jsx-3408573263"]').text
    for j in elementos:
        
        nombres = j.find_element_by_xpath('.//td[@class="jsx-428502957 property-name"]').text
        valores = j.find_element_by_xpath('.//td[@class="jsx-428502957 property-value"]').text
        nombre.append(nombres)
        valor.append(valores)
        
    if ("Nombre comercial" in nombre)==True:
        index_modelo = nombre.index("Nombre comercial")
        modelo.append(valor[index_modelo].upper())
    
    if ("Nombre comercial" in nombre)==False:
        modelo.append("Modelo no encontrado")
    
    if("Procesador" in nombre)==True:
        index_procesador = nombre.index("Procesador")
        procesador.append(valor[index_procesador].upper())
        
    if("Procesador" in nombre)==False:
        procesador.append("Procesador no encontrado")
        
    if("Unidad de estado sólido SSD" in nombre)==True:
        index_almacenamiento = nombre.index("Unidad de estado sólido SSD")
        almacenamiento.append(valor[index_almacenamiento].upper())
        
    if("Unidad de estado sólido SSD" in nombre)==False:
        almacenamiento.append("Almacenamiento no encontrado")
        
    if("Memoria RAM" in nombre)==True:
        index_ram= nombre.index("Memoria RAM")
        ram.append(valor[index_ram].upper())
        
    if("Memoria RAM" in nombre)==False:
        ram.append("Memoria Ram no encontrada") 
        
    
    precios.append(precio)
    codigo.append(codigos)
    nombre_equipo.append(nombre_aux)
        

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

for i in range(len(codigo)):
    codigo[i] = codigo[i].replace("Código del producto: ","")


x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i],'Codigo':codigo[i],'Modelo':modelo[i] ,'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], "Fecha":x},ignore_index=True)

In [12]:
modelo

['Modelo no encontrado',
 'LENOVO IDEAPAD 3',
 'Modelo no encontrado',
 'NOTEBOOK HP ENVY 13-BA1124LA',
 'Modelo no encontrado',
 'ASUS TUF GAMING F15 FX506LH-HN110T',
 'NOTEBOOK HP 348 G7',
 'NOTEBOOK PAVILION GAMING 15-EC1026LA AMD RYZEN 5 8GB 256GB SSD NVIDIA GEFORCE GTX 1650',
 'ASPIRE 5 A515-54-34VM-2',
 'Modelo no encontrado',
 'ASUS VIVOBOOK 14 X412FA-EB1261TS',
 'NITRO 5 AN515-55-56P2-1',
 'HP PAVILION GAMING 15-DK1044LA',
 'MACBOOK AIR (M1, 2020)',
 'Modelo no encontrado',
 'MACBOOK AIR (M1, 2020)',
 'ASUS TUF GAMING F15 FX506LH-HN155T',
 'ASUS ROG ZEPHYRUS G14 GA401QM-K2120T',
 'MACBOOK AIR (M1, 2020)',
 'ASUS LAPTOP X515MA-BR576T',
 'Modelo no encontrado',
 'MACBOOK AIR (M1, 2020)',
 'MACBOOK PRO (13 PULGADAS, MI, 2020)',
 'MACBOOK PRO (13 PULGADAS, MI, 2020)',
 'Modelo no encontrado',
 'Modelo no encontrado',
 'MACBOOK PRO (13 PULGADAS, MI, 2020)',
 'ASUS ZENBOOK 13 UX325EA-KG325T',
 'MACBOOK PRO (13 PULGADAS, MI, 2020)',
 'NOTEBOOK HP PAVILION 15-EH0010LA',
 'Modelo no enc

HP

In [13]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,6):
    url = "https://www.hp.com/cl-es/shop/notebooks/notebooks-intel.html?gclsrc=aw.ds&p="+str(i)
    url_base.append(url)

##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//li[@class="item product product-item"]')
    for i in notebook:
        try:
            link = i.find_element_by_tag_name('a').get_attribute('href')
        except:
            x = 2
        url_notebooks.append(link)

##Listas que almacenaran datos encontrados dentro del sitio web
marca = []
nombre_equipo = []
modelo = []
procesador = []
almacenamiento = []
codigo = []
ram = []
precios = []

for i in url_notebooks:
    nombres = []
    valores = []
    sleep(2)
    driver.get(i)
    try:
        boton = driver.find_element_by_xpath('//div[@class="data item title" and @id="tab-label-additional"]').click()
    except:
        try:

            boton = driver.find_element_by_xpath('//a[@class="data switch" and @data-toggle="trigger"').click()
        except:
            x = "boton no encontrado"
    sleep(5)
    try:
        name = driver.find_element_by_xpath('//span[@class="base" and @data-ui-id="page-title-wrapper"]').text
    except:
        name = "Nombre no encontrado"
    try:
        price = driver.find_element_by_xpath('//span[@class="price"]').text
    except:
        price = "Precio no encontrado"
    
    try:
        code = driver.find_element_by_xpath('//div[@class="value " and @itemprop="sku"]').text
    except:
        code = "Codigo no encontrado"
    bloque = driver.find_elements_by_xpath('.//tr[@class="item"]')
    for j in bloque:
        nombre = j.find_element_by_xpath('.//th[@class="col label"]').text
        valor = j.find_element_by_xpath('.//td[@class="col data"]').text
        nombres.append(nombre)
        valores.append(valor)
        
    if ("Procesador" in nombres)==True:
        index_procesador = nombres.index("Procesador")
        procesador.append(valores[index_procesador])
        
    elif ("Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")
        
    if ("Memoria, estándar" in nombres)==True:
        index_ram = nombres.index("Memoria, estándar")
        ram.append(valores[index_ram])
        
    elif ("Memoria, estándar" in nombres)==False:
        ram.append("Memoria ram no especificada")
        
    if("Descripción del disco duro" in nombres)==True:
        index_almacenamiento = nombres.index("Descripción del disco duro")
        almacenamiento.append(valores[index_almacenamiento])
    
    elif("Descripción del disco duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")     
    
    
        
    nombre_equipo.append(name)
    precios.append(price)
    codigo.append(code)

for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")
    if(precios[i]==""):
        precios[i] = "Precio no especificado"
        
for i in range(len(nombre_equipo)):
    modelo.append(nombre_equipo[i].replace("Notebook HP", ""))
    

In [14]:
#Llenado de cada uno de los campos
x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i], 'Codigo':codigo[i], 'Modelo':modelo[i],'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], 'Precio':precios[i], "Fecha":x},ignore_index=True)

In [15]:
modelo

[' 240 G8',
 ' ZBook Power G7',
 ' ZBook Firefly 14 G8',
 ' Pavilion x360 Convertible 14-dy0501la',
 ' Pavilion Gaming 15-dk1033la',
 ' ZBook Firefly 15,6 G8',
 ' 14-dq2026la',
 ' Spectre x360 Convertible 14-ea0001la',
 ' Pavilion x360 Convertible 14-dy0002la',
 ' Pavilion Gaming 15-dk1037la',
 ' 15-dw1064la',
 ' 14-cf2051la',
 ' 240 G7',
 ' 348 G7',
 ' Pavilion 13-bb0004la',
 ' 15-ef1015la',
 'Notebook OMEN 15-ek0062la',
 ' Pavilion Gaming - 15-dk1028la',
 ' 14-cf2053la',
 ' 15-da2031la',
 ' EliteBook x360 1040 G6',
 ' ZBook Firefly 14 G7',
 ' ZBook Firefly 14 G7',
 ' 250 G8',
 ' Pavilion Gaming 15-dk1043la',
 ' 14-ck2096la',
 ' 15-dw2039la',
 ' 15-dy1064la',
 ' Pavilion x360 Convertible 15-dq1003la',
 ' 14-ck2091la',
 ' ENVY 13-ba1124la',
 ' 348 G7',
 ' ZBook Power G7',
 ' Pavilion x360 Convertible 14-dh1032la',
 ' 15-da2034la',
 ' 14-cf3028la',
 ' 14-cf2087la',
 ' Pavilion x360 Convertible 14-dw0005la',
 ' Pavilion 14-dv0001la',
 ' ENVY 13-ba1123la',
 ' 14-dq1016la',
 ' 15-dw2043la'

Ripley


In [16]:
##Obtención de urls de cada una de las pagina del sitio web para la navegación de notebooks
url_base=[]
for i in range(1,8):
    url = "https://simple.ripley.cl/tecno/computacion/notebooks?page="+str(i)
    url_base.append(url)

##Obtención de urls de cada uno de los notebooks del sitio para la posterior navegación en cada uno de ellos
url_notebooks=[]
for url in url_base:
    driver.get(url)
    notebook = driver.find_elements_by_xpath('//div[@class="catalog-product-item catalog-product-item__container col-xs-6 col-sm-6 col-md-4 col-lg-4"]')
    for i in notebook:
        link = i.find_element_by_tag_name('a').get_attribute('href')
        url_notebooks.append(link)

##Obtención de datos dentro de cada una de las páginas del sitio web, almacenados en listas auxiliares

nombre_equipo = []
precios = []
almacenamiento = []
procesador = []
ram = []
modelo = []
codigos = []

for i in url_notebooks:
    nombres = []
    valores = []        
    driver.get(i)
    sleep(5)
    try:
        boton = driver.find_element_by_xpath('//div[@aria-controls="panel-Especificaciones"]').click()
    except:
        boton = "holi"
    sleep(2)
    try:
        ver = driver.find_element_by_xpath('//button[@class="btn-loading  rpl-show-more btn-outline-grey "]').click()
    except:
        x=2
    bloque = driver.find_elements_by_xpath('//tr')
    bloque_nombre = driver.find_elements_by_xpath('//section[@class="product-header hidden-xs"]')
    try:
        precio = driver.find_element_by_xpath('//dt[@class="product-price"]')
    except:
        try:
            precio = driver.find_element_by_xpath('//dt[@class="product-price product-price__line-thru"]')
        except:
            precio = "Precio no encontrado"
    for i in bloque_nombre:
        nombre = i.find_element_by_xpath('.//h1')
        codigo = i.find_element_by_xpath('.//span[@class="sku sku-value"]')
    for i in bloque:
        elementos = i.find_elements_by_xpath('.//td')
        for j in range(len(elementos)):
            if (j==0):
                nombres.append(elementos[j].text)
            if (j==1):
                valores.append(elementos[j].text)
                
    if ("Tipo Procesador" in nombres)==True:
        index_procesador = nombres.index("Tipo Procesador")
        procesador.append(valores[index_procesador])

    if ("Tipo Procesador" in nombres)==False:
        procesador.append("Procesador no especificado")

    if ("Memoria Ram" in nombres)==True:
        index_ram = nombres.index("Memoria Ram")        
        ram.append(valores[index_ram])
        
    if ("Memoria Ram" in nombres)==False:
        ram.append("Memoria Ram no especificada")
        
    if ("Capacidad Disco Duro" in nombres)==True:
        idex_almacenamiento = nombres.index("Capacidad Disco Duro")
        almacenamiento.append(valores[idex_almacenamiento])
        
    if ("Capacidad Disco Duro" in nombres)==False:
        almacenamiento.append("Almacenamiento no especificado")
        
    if ("Modelo" in nombres)==True:
        index_modelo = nombres.index("Modelo")
        modelo.append(valores[index_modelo])
        
    if ("Modelo" in nombres)==False:
        modelo.append("Modelo no especificado")
        
    codigos.append(codigo.text)
    try:
        precios.append(precio.text)

    except:
        precios.append(precio)
    nombre_equipo.append(nombre.text)


for i in range(len(precios)):
    precios[i] = precios[i].replace(" ", "")
    precios[i] = precios[i].replace("$", "")
    precios[i] = precios[i].replace(".", "")

x = datetime.today().strftime('%Y-%m-%d')
for i in range(len(url_notebooks)):
    df = df.append({'Nombre': nombre_equipo[i], 'Codigo':codigos[i],'Modelo':modelo[i],'Almacenamiento':almacenamiento[i],'Procesador':procesador[i], 'Memoria Ram':ram[i], "Precio":precios[i], "Fecha":x},ignore_index=True)

In [17]:
modelo

['TUF DASH F15 FX516',
 'TUF GAMING F15 FX506LH',
 'PAVILION X360 CONVERTIBLE 14-DY0501LA',
 'LAPTOP 14-DQ2026LA',
 'E410MA-EK746T',
 '15-EC1026LA',
 'UM425UAZ-KI021T',
 'Yoga C940-14IIL',
 'IdeaPad Flex 5',
 'E410MA-EK1016T',
 '15-EC1024LA',
 'Nitro 5',
 'C733',
 '348 G7',
 'Legion 5',
 'Air M1',
 'UM425UA',
 'X515EA-BQ968T',
 'MAGICBOOK',
 'Yoga C740',
 'Yoga Slim 9I',
 'IDEAPAD S540',
 'Inspiron 3505',
 '250 G8',
 'ROG ZEPHYRUS G14 GA401QC',
 'Modelo no especificado',
 'Nitro 5',
 '13-BB0004LA',
 'Aspire 3',
 '348 G7',
 'CHROMEBOOK',
 'Ideapad 5 Pro',
 'ROG Zephyrus G14',
 'LAPTOP 14-CF2051LA',
 'Yoga 9I',
 'Spin 3',
 'Ideapad Gaming 3',
 'ZENBOOK 13 OLED UX325EA',
 'Ideapad 5',
 'PAVILION X360 CONVERTIBLE 15-ER0521LA',
 '1 14ADA05',
 'Inspiron 15 3501',
 '16-A0004LA',
 'PAVILION X360 CONVERTIBLE 14-DY0002LA',
 'Spin 3',
 'UM325SA',
 'Inspiron 5310',
 'Inspiron 5415',
 'Aspire 3',
 'Zbook Firefly 14 G7',
 'Nitro 5',
 'IDEAPAD 5 PRO',
 'PAVILION LAPTOP 15-EH0011LA',
 'Matebook D14',


In [19]:
df.to_csv('NotebooksOficial.csv')


In [20]:
df

,0,Almacenamiento,Codigo,Fecha,Memoria Ram,Modelo,Nombre,Precio,Procesador
0,Precio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Codigo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nombre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Procesador,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fecha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
825,NaN,Almacenamiento no especificado,MPM00033460068,2021-09-29,32GB,Latitude 7490,"NOTEBOOK DELL LATITUDE 7490 14”, CORE I5, 32GB...",849990,Procesador no especificado
826,NaN,Almacenamiento no especificado,MPM00033460102,2021-09-29,16GB,TECRA X40-D,"NOTEBOOK TOSHIBA TECRA X40-D 14”, CORE I7, 16G...",699990,Procesador no especificado
827,NaN,Almacenamiento no especificado,MPM00033513636,2021-09-29,8GB,LATITUDE 7490,"NOTEBOOK DELL LATITUDE 7490 14”, CORE I5, 8GB ...",749990,Procesador no especificado
828,NaN,Almacenamiento no especificado,MPM00032935486,2021-09-29,Memoria Ram no especificada,Dell Gamer G3 15-3500,DELL GAMER G3 15-3500 16GB RAM 1TB SSD REACOND...,1249990,Procesador no especificado


In [ ]:
len(nombre_equipo)

285

101

[]

In [33]:
df

,0,Almacenamiento,Codigo,Fecha,Memoria Ram,Modelo,Nombre,Precio,Procesador
0,Precio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Codigo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nombre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Procesador,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fecha,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
73,NaN,256 Gb,851769001,2021-09-29,8 Gb,Asus Vivobook 14 X413ea-eb669,Notebook Asus Vivobook 14 X413ea-eb669t,799990,Intel Core I5
74,NaN,256 Gb,867618001,2021-09-29,8 Gb,Msi Modern 14 B10mw-258c,Notebook Msi Modern 14 B10mw-258cl,909990,Intel Core I5
75,NaN,256 Gb,857109001,2021-09-29,8 Gb,Dell G5 15 550,Notebook Dell G5 15 5500,1249990,Intel Core I7
76,NaN,512 Gb,857358001,2021-09-29,16 Gb,Gamer Acer Nitro 5 An515-55-56p2-,Notebook Gamer Acer Nitro 5 An515-55-56p2-2,899990,Intel Core I5


In [43]:
df.columns

Index(['Precio', 'Codigo', 'Nombre', 'Procesador', 'Fecha', 'Memoria Ram',
       'Almacenamiento', 'Modelo'],
      dtype='object')

In [42]:
df

,Precio,Codigo,Nombre,Procesador,Fecha,Memoria Ram,Almacenamiento,Modelo
0,299990,857128001,Notebook Reacondicionado Hp 14-fq0013dx,Amd Athlon,2021-09-29,4 Gb,128 Gb,Reacondicionado Hp 14-fq0013d
1,549990,866158001,Notebook Lenovo Ideapad 3 14alc6,Amd Ryzen 5,2021-09-29,8 Gb,256 Gb,Lenovo Ideapad 3 14alc
2,769990,861708001,Notebook Gamer Asus Tuf Gaming Fx506lh-hn082t,Intel Core I5,2021-09-29,8 Gb,512 Gb,Gamer Asus Tuf Gaming Fx506lh-hn082
3,399990,841656001,Notebook Lenovo Ideapad 3,Amd Athlon,2021-09-29,8 Gb,1 Tb,Lenovo Ideapad
4,419990,861707001,Notebook Asus X515ea-bq968t,Intel Core I3,2021-09-29,4 Gb,128 Gb,Asus X515ea-bq968
...,...,...,...,...,...,...,...,...
65,799990,851769001,Notebook Asus Vivobook 14 X413ea-eb669t,Intel Core I5,2021-09-29,8 Gb,256 Gb,Asus Vivobook 14 X413ea-eb669
66,909990,867618001,Notebook Msi Modern 14 B10mw-258cl,Intel Core I5,2021-09-29,8 Gb,256 Gb,Msi Modern 14 B10mw-258c
67,1249990,857109001,Notebook Dell G5 15 5500,Intel Core I7,2021-09-29,8 Gb,256 Gb,Dell G5 15 550
68,899990,857358001,Notebook Gamer Acer Nitro 5 An515-55-56p2-2,Intel Core I5,2021-09-29,16 Gb,512 Gb,Gamer Acer Nitro 5 An515-55-56p2-
